In [1]:
import pandas as pd
import numpy as np
import jieba
import tensorflow as tf
import re
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
reviews = pd.read_csv("data.csv")
reviews.columns = ['summarization','article']
reviews.head(100)

,summarization,article
0,深圳地铁将设立ＶＩＰ头等车厢 买双倍票可享坐票,南都讯 记者刘凡 周昌和 任笑一 继推出日票后，深圳今后将设地铁ＶＩＰ头等车厢，设坐票制。昨...
1,中国西部是地球上主要干旱带之一，妇女是当地劳动力．．．,同心县地处宁夏中部干旱带的核心区， 冬寒长，春暖迟，夏热短，秋凉早，干旱少雨，蒸发强烈，风大...
2,思源焦点公益基金救助孩子：永康,不满一岁的永康是个饱经病痛折磨的孩子，２０１１年７月５日出生的他，患有先天性心脏病、疝气，一...
3,康师傅回应转卖废弃茶叶：下家承诺用废料做枕头,就废弃茶叶被转手事件发声明本报讯（记者刘俊） “我们也是受害者！”昨日，有媒体报道称康师傅...
4,活动时间：,·奖励办法：率先提交的前１００个创意项目，经评估，可优先资助实施。·咨询电话：０１０－６７...
5,５．１２灾后重建资助项目投票评选,２００９年８月，《２００９中国慈善导航行动》第一季正式启动，此档由ＣＣＴＶ－１２《大家看法》...
6,以书为友，知行合一—２０１２年小桔灯湖北站,２０１２年东风标致小桔灯乡村小学图书馆计划于６月２３日－２９日在湖北省武汉市新洲区凤凰镇郭岗...
7,博爱周活动时间：,１、纪念汶川地震１周年２、纪念５月１２日国家首个“防灾减灾日”３、纪念索尔弗利诺战役１５...
8,“金葵花”羌族少儿合唱团公益活动,２、９月２２日，与阿坝州及茂县各方人士商议在四川省阿坝藏族羌族自治州组建“羌族少儿合唱团”...
9,视频征集入围短片,从汽车发明到现在，平均每天因事故死亡３３００人左右，相当于１０架大型客机坠毁。这些事故中，９...


In [3]:
# import json
# source_file = './data/evaluation_with_ground_truth.txt'
# reviews = pd.DataFrame.from_dict({"summarization":"","article":"","index":""},orient='index').T
# with open(source_file, 'r', encoding='utf-8') as f:
#     source_data = f.read()
# i = 0
# for line in source_data.split('\n'):
#     line = eval(line)
#     reviews.loc[i] = line
#     i = i + 1
#     if i == 2000:
#         break
# reviews.head()

In [4]:
# reviews = pd.read_table('./data/evaluation_with_ground_truth.txt',names=['summarization','article','index'],encoding='utf-8')
reviews.isnull().sum()
reviews = reviews.dropna()
# reviews = reviews.drop(['index'], 1)
reviews = reviews.reset_index(drop=True)
reviews.head()

,summarization,article
0,深圳地铁将设立ＶＩＰ头等车厢 买双倍票可享坐票,南都讯 记者刘凡 周昌和 任笑一 继推出日票后，深圳今后将设地铁ＶＩＰ头等车厢，设坐票制。昨...
1,中国西部是地球上主要干旱带之一，妇女是当地劳动力．．．,同心县地处宁夏中部干旱带的核心区， 冬寒长，春暖迟，夏热短，秋凉早，干旱少雨，蒸发强烈，风大...
2,思源焦点公益基金救助孩子：永康,不满一岁的永康是个饱经病痛折磨的孩子，２０１１年７月５日出生的他，患有先天性心脏病、疝气，一...
3,康师傅回应转卖废弃茶叶：下家承诺用废料做枕头,就废弃茶叶被转手事件发声明本报讯（记者刘俊） “我们也是受害者！”昨日，有媒体报道称康师傅...
4,活动时间：,·奖励办法：率先提交的前１００个创意项目，经评估，可优先资助实施。·咨询电话：０１０－６７...


In [5]:
def clean_text(text,remove_stopwords=True):
#     if True:
#         text = text.split()
#         new_text = []
#         for word in text:
#             if word in contractions:
#                 new_text.append(contractions[word])
#             else:
#                 new_text.append(word)
#         text = " ".join(new_text)
    
    text = re.sub(r'<Paragraph>', '', text) 
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', '', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', '', text)
    text = re.sub(r'<br />', '', text)
    text = re.sub(r'\'', '', text) 
    
    text = jieba.cut(text)
    if text != '\r\n': #换行符
        text = " ".join(text)
    
#     text_list = list(text)
#     text = " ".join(text_list)
    
            
    if remove_stopwords:
        stopwords = pd.read_csv("stopwords.txt",index_col=False,sep="\t",quoting=3,names=['stopword'], encoding='utf-8')    
        stopwords = stopwords.stopword.values.tolist()
        text = text.split()
        text = [w for w in text if not w in stopwords]
        text = " ".join(text)
        
    return text

In [6]:
clean_summaries = []
for summary in reviews.summarization:
    clean_summaries.append(clean_text(summary,remove_stopwords=False))
clean_texts = []
for text in reviews.article:
    clean_texts.append(clean_text(text))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 0.574 seconds.
Prefix dict has been built succesfully.


KeyboardInterrupt: 

In [ ]:
for i in range(0,20):
    print("Clean Review #",i+1)
    print(clean_summaries[i])
    print(clean_texts[i])
    print()

In [ ]:
def count_words(count_dict,text):
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [ ]:
word_counts = {}
count_words(word_counts,clean_summaries)
count_words(word_counts,clean_texts)
print("Size of Vocabulary:", len(word_counts))

In [ ]:
path = 'sgns.sogounews.bigram-char'
topn = 100
def read_vectors(path, topn):  # read top n word vectors, i.e. top is 10000
    lines_num, dim = 0, 0
    vectors = {}
    iw = []
    wi = {}
    with open(path, encoding='utf-8', errors='ignore') as f:
        first_line = True
        for line in f:
            if first_line:
                first_line = False
                dim = int(line.rstrip().split()[1])
                continue
            lines_num += 1
            tokens = line.rstrip().split(' ')
            vectors[tokens[0]] = np.asarray([float(x) for x in tokens[1:]])
            iw.append(tokens[0])
            if topn != 0 and lines_num >= topn:
                break
    for i, w in enumerate(iw):
        wi[w] = i
    return vectors, iw, wi, dim
embeddings_index, iw, wi, dim = read_vectors(path, topn)

In [ ]:
missing_words = 0
threshold = 2

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

In [ ]:
vocab_to_int = {}
value = 0
for word,count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value +=1
        
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)    
    
# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

In [ ]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

In [ ]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [ ]:
# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

In [ ]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [ ]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

In [ ]:
# Inspect the length of texts
print(np.percentile(lengths_texts.counts, 90))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))

In [ ]:
# Inspect the length of summaries
print(np.percentile(lengths_summaries.counts, 90))
print(np.percentile(lengths_summaries.counts, 95))
print(np.percentile(lengths_summaries.counts, 99))

In [ ]:
def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [ ]:
# Sort the summaries and texts by the length of the texts, shortest to longest
# Limit the length of summaries and texts based on the min and max ranges.
# Remove reviews that include too many UNKs

sorted_summaries = []
sorted_texts = []
max_text_length = 6200
max_summary_length = 40
min_length = 2
# unk_text_limit = 1
# unk_summary_limit = 0
# print (int_summaries)
for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
#             unk_counter(int_summaries[count]) <= unk_summary_limit and
#             unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

In [ ]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length

In [ ]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [ ]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    '''Create the encoding layer'''
    
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
    # Join outputs since we are using a bidirectional RNN
    enc_output = tf.concat(enc_output,2)
    
    return enc_output, enc_state

In [ ]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_summary_length):
    '''Create the training logits'''
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       initial_state,
                                                       output_layer) 

    training_logits,_,_  = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_logits

In [ ]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_summary_length, batch_size):
    '''Create the inference logits'''
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        initial_state,
                                                        output_layer)
                
    inference_logits,_,_  = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_logits

In [ ]:
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length, 
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    
    for layer in range(num_layers):
        with tf.variable_scope('decoder_{}'.format(layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                     input_keep_prob = keep_prob)
    # creating Dense- This is also called output layer. This will produce the summary.
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    # Using BahdanauAttention as one of the widely used Attention Algorithms
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  text_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')

    dec_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell,
                                                          attn_mech,
                                                          rnn_size)
    
    # initializing the initial state, layer it would be update by output from one cell        
    initial_state = dec_cell.zero_state(batch_size=batch_size, dtype=tf.float32)
    # Creating training logits - which would be used during training dataset
    with tf.variable_scope("decode"):
        training_logits = training_decoding_layer(dec_embed_input, 
                                                  summary_length, 
                                                  dec_cell, 
                                                  initial_state,
                                                  output_layer,
                                                  vocab_size, 
                                                  max_summary_length)
    # Creating inference logits - which would produce output using train model
    with tf.variable_scope("decode", reuse=True):
        inference_logits = inference_decoding_layer(embeddings,  
                                                    vocab_to_int['<GO>'], 
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell, 
                                                    initial_state, 
                                                    output_layer,
                                                    max_summary_length,
                                                    batch_size)

    return training_logits, inference_logits

In [ ]:
def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    '''Use the previous functions to create the training and inference logits'''
    
    # Use Numberbatch's embeddings and the newly created ones as our embeddings
    embeddings = word_embedding_matrix
    
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    
    return training_logits, inference_logits

In [ ]:
def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [ ]:
def get_batches(summaries, texts, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths

In [ ]:
# Set the Hyperparameters
epochs = 100
batch_size = 64
rnn_size = 256
num_layers = 2
learning_rate = 0.005
keep_probability = 0.75

In [ ]:
# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size)
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits.rnn_output, 'logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')
    
    # Create the weights for sequence_loss
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph is built.")

In [46]:
# Subset the data for training
start = 0
end = start + 1500
sorted_summaries_short = sorted_summaries[start:end]
sorted_texts_short = sorted_texts[start:end]
print("The shortest text length:", len(sorted_texts_short[0]))
print("The longest text length:",len(sorted_texts_short[-1]))

The shortest text length: 5
The longest text length: 354


In [47]:
# Train the Model
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 20 # Check training loss after every 20 batches
stop_early = 0 
stop = 4 # If the update loss does not decrease in 4 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(sorted_texts_short)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model

checkpoint = "./best_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
#     loader = tf.train.import_meta_graph("./" + checkpoint + '.meta')
#     loader.restore(sess, checkpoint)
    
    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries_short, sorted_texts_short, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts_short) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0

            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break

Average loss for this update: 11.409
New Record!
Average loss for this update: 6.175
New Record!
Average loss for this update: 6.085
New Record!
Epoch   1/100 Batch   20/23 - Loss:  7.688, Seconds: 16.44
Average loss for this update: 6.27
No Improvement.
Average loss for this update: 5.244
New Record!
Average loss for this update: 5.35
No Improvement.
Epoch   2/100 Batch   20/23 - Loss:  5.575, Seconds: 15.88
Average loss for this update: 5.701
No Improvement.
Average loss for this update: 4.966
New Record!
Average loss for this update: 5.084
No Improvement.
Epoch   3/100 Batch   20/23 - Loss:  5.207, Seconds: 15.88
Average loss for this update: 5.249
No Improvement.
Average loss for this update: 4.69
New Record!
Average loss for this update: 4.868
No Improvement.
Epoch   4/100 Batch   20/23 - Loss:  4.903, Seconds: 16.48
Average loss for this update: 4.896
No Improvement.
Average loss for this update: 4.42
New Record!
Average loss for this update: 4.645
No Improvement.
Epoch   5/100 B

In [48]:
def text_to_seq(text):
    '''Prepare the text for the model'''
    
    text = clean_text(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

In [52]:
# Create your own review or use one from the dataset
#input_sentence = "I have never eaten an apple before, but this red one was nice. \
                  #I think that I will try a green apple next time."
#text = text_to_seq(input_sentence)
random = np.random.randint(0,len(clean_texts))
input_sentence = clean_texts[1006]
text = text_to_seq(clean_texts[1006])

checkpoint = "./best_model.ckpt"

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    #Multiply by batch_size to match the model's input parameters
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(5,8)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 

# Remove the padding from the tweet
pad = vocab_to_int["<PAD>"] 

print('Original Text:', input_sentence)

print('\nText')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

print('\nSummary')
print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from ./best_model.ckpt
Original Text: 福州 新闻 海都网 环卫工人 节日 享用 海鲜大餐 志华摄 昨天 五一劳动节 劳动者 节日 昨天中午 扬州 市区 一家 名叫 海盗 船长 平价 海鲜 饭店 老板 蒋军 邀请 邗江 环卫所 名 一线 环卫工人 免费 吃 海鲜大餐 方式 城市 美容师 致敬 招待 这批 环卫 贵宾 蒋 老板 婉拒 前去 就餐 客人 环卫工 第一次 走进 海鲜 酒店 昨天上午 点 海盗 船长 饭店 一楼 大厅 墙上 挂 热烈欢迎 环卫工 师傅 莅临 海盗 船长 用餐 横幅 乖乖 热烈欢迎 环卫工 师傅 请客 饭店 老板 环卫工 尊重 字 热乎乎 岁 环卫工 马五喜 说出 感受 饭店 大厅 环卫工 包场 中心 位置 放上 张大 圆桌 招待 环卫工 大厅 南墙 零星 卡座 接待 散客 上菜 糖醋排骨 鸭肫 时鲜 毛豆 道 冷菜 桌 啤酒 饮料 自选 倒 饮料 啤酒 举起 杯子 庆祝 节日 油淋 大虾 蒜 茸 粉丝 蒸 十头 鲍 家烧 缩骨鱼 辣 炒 花甲 姜葱 炒 蛏子 文蛤 蒸 鸡蛋 道 热菜 点心 精美 水果 拼盘 大伙 一饱口福 绝大多数 环卫工 这是 平生 第一次 进 海鲜 大酒店 吃饭 天前 接到 所长 关照 吃饭 通知 激动 想不到 干 环卫 受 尊重 小常 母亲 告诉 干 环卫 第一次 收到 邀请 环卫工 辛苦 总想 做点 劳动节 当天 邀请 名 环卫工 吃饭 一点 自我 炒作 想法 表达 尊重 平时 起早贪黑 扫 马路 实在 辛苦 老板 蒋军 介绍 平时 路上 开车 环卫工 天不亮 扫 马路 捡 马路 中央 垃圾 穿梭 车流 中 辛苦 危险 做点 想法 烟花 三月 经贸 旅游节 前 扬州 中外 嘉宾 打造 洁净 市容 环境 白天 忙碌 外 深夜 路面 车流 少时 冲洗 马路 触动 劳动节 期间 请 环卫工 吃饭 劳动节 当天 酒店 忙 避开 劳动节 当天 请 劳动节 当天 请 表达 环卫工 敬意 折扣 婉拒 客人 得知 请 环卫工 吃饭 理解 环卫工 吃 酒店 工作人员 环卫工 热情服务 菜谱 每桌 1800 元 标准 安排 标准 酒店 中 高档次 环卫工 吃饭 真的 这位 老板 爱心 感动 环卫工 相互 敬酒 热闹 场景 坐在 